In [2]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")
import time

In [3]:
#date_range

In [4]:
workouts = pd.read_csv('workouts.csv')
workouts = workouts[workouts.user_id!=2509]

In [5]:
def epoch_end_convert(df):
    '''
    takes dataframe and converts date column to epoch milliseconds
    inputs:
        df = dataframe name
    '''
    try:
        return (int(time.mktime(time.strptime(df['end'], '%Y-%m-%d %H:%M:%S'))) - 14400)*1000
    except:
        #handles activities that have fractions of a second included
        return (int(time.mktime(time.strptime(df['end'][:19], '%Y-%m-%d %H:%M:%S'))) - 14400)*1000

In [6]:
#workouts_subset['end_epoch'] = workouts_subset.apply(epoch_end_convert, axis=1)
workouts['end_epoch'] = workouts.apply(epoch_end_convert, axis=1)

In [7]:
workouts.head()

,user_id,start,end,sport_id,name,intensity_score,z1,z2,z3,z4,z5,end_epoch
0,828,2015-09-15 20:08:00,2015-09-15 22:56:00,35,Track & Field,20.508564,1066,1810,1432,4668,679,1442357760000
1,828,2015-09-19 12:46:00,2015-09-19 14:09:00,35,Track & Field,16.823285,506,601,1861,696,22,1442671740000
2,828,2015-09-20 13:15:00,2015-09-20 16:00:00,35,Track & Field,20.506562,929,1302,1329,5353,321,1442764800000
3,828,2015-09-21 18:55:34.582,2015-09-21 18:58:37.876,1,Cycling,4.278678,8,170,5,0,0,1442861917000
4,828,2015-09-21 19:19:00,2015-09-21 21:12:00,35,Track & Field,19.176787,655,1757,1372,2781,28,1442869920000


#Adding in Time Periods (For Either Weeks or Races)

In [8]:
date_markers = pd.read_csv('epoch_dates.csv')

In [9]:
date_markers.head()

,ET_Date,date_start_epoch,race_period,bike_period,race_period_start,race_period_end,bike_period_start,bike_period_end
0,9/15/2015 0:00,1442289600000,1,1,1,0,1,0
1,9/16/2015 0:00,1442376000000,1,1,0,0,0,0
2,9/17/2015 0:00,1442462400000,1,1,0,0,0,0
3,9/18/2015 0:00,1442548800000,1,1,0,0,0,0
4,9/19/2015 0:00,1442635200000,1,1,0,0,0,0


In [10]:
date_markers = date_markers[date_markers['race_period_start'] > 0].reset_index()
#dictionary of race period to start epoch
race_start_dict = {}
for i in range(0, len(date_markers)):
    race_start_dict[date_markers.values[i][3]] = date_markers.values[i][2]

#date_markers = date_markers[date_markers['bike_period_start'] > 0].reset_index()
#bike_start_dict = {}
#for i in range(0, len(date_markers)):
#    bike_start_dict[date_markers.values[i][4]] = date_markers.values[i][2]


In [11]:
race_start_dict

{1L: 1442289600000L,
 2L: 1443240000000L,
 3L: 1445054400000L,
 4L: 1446264000000L,
 5L: 1447473600000L}

In [12]:
#bike_start_dict

In [13]:
def add_race(df):
    if (df.end_epoch >= race_start_dict[1]) and (df.end_epoch < race_start_dict[2]):
        return 1
    elif (df.end_epoch >= race_start_dict[2]) and (df.end_epoch < race_start_dict[3]):
        return 2
    elif (df.end_epoch >= race_start_dict[3]) and (df.end_epoch < race_start_dict[4]):
        return 3
    elif (df.end_epoch >= race_start_dict[4]) and (df.end_epoch < race_start_dict[5]):
        return 4
    elif (df.end_epoch >= race_start_dict[5]) and (df.end_epoch < 1448164800000):
        return 5
    else:
        return 0


In [14]:
def add_bike(df):
    if (df.end_epoch >= bike_start_dict[1]) and (df.end_epoch < bike_start_dict[2]):
        return 1
    elif (df.end_epoch >= bike_start_dict[2]) and (df.end_epoch < bike_start_dict[3]):
        return 2
    elif (df.end_epoch >= race_start_dict[3]) and (df.end_epoch < bike_start_dict[4]):
        return 3
    elif (df.end_epoch >= bike_start_dict[4]) and (df.end_epoch < bike_start_dict[5]):
        return 4
    elif (df.end_epoch >= bike_start_dict[5]) and (df.end_epoch < bike_start_dict[6]):
        return 5
    elif (df.end_epoch >= bike_start_dict[6]) and (df.end_epoch < bike_start_dict[7]):
        return 6
    elif (df.end_epoch >= bike_start_dict[7]) and (df.end_epoch < bike_start_dict[8]):
        return 7
    elif (df.end_epoch >= bike_start_dict[8]) and (df.end_epoch < bike_start_dict[9]):
        return 8
    elif (df.end_epoch >= bike_start_dict[9]):
        return 9

In [49]:
def add_periods(date_markers, df, buildup_days=0, version=None):
    '''
    Adds race periods to the input dataframe 'df'
        Inputs: 
            date_markers: dataframe with the ET date, the start epoch time of that date, and which race period the date is in
            df: the dataframe to add these date markers to
            buildup_days: if 0, we use the entire time period before the race/bike, after the previous one
                if not 0, and instead a value "n", we look at the "n" days before the race/bike as the "period"
            version: takes two values: "bike", or "race", and treats appropriately
    '''
    if version==None:
        print "Enter Either 'race' or 'bike'!"
    if buildup_days==0:
        if version=="race":
            #Collapse the data frame into only the start and end dates for each race period
            date_markers = date_markers[date_markers['race_period_start'] > 0].reset_index()
            
            #dictionary of race period to start epoch
            race_start_dict = {}
            for i in range(0, len(date_markers)):
                race_start_dict[date_markers.values[i][3]] = date_markers.values[i][2]
            
            #Now label each race period as 1, 2, 3, 4, or 5
            df['race_period'] = df.apply(add_race, axis=1)
            
            #gett the month and day and put in a column
            df['date_md']=df['end'].str[5:10]
            df=df.sort(['user_id', 'end_epoch', 'race_period'], ascending=[True, True, False])
            
            #Create a column for the number of activites for a given day
            groups = df.groupby(['user_id','date_md'])
            things = groups.agg(['count']).join(pd.DataFrame(groups.size(),columns=['activity_count']))
            count_df = pd.DataFrame(things).reset_index()
            count_df = count_df.drop([count_df.columns[2], count_df.columns[3], count_df.columns[4], count_df.columns[5],
                count_df.columns[6], count_df.columns[7], count_df.columns[8], count_df.columns[9], count_df.columns[10],
                count_df.columns[11], count_df.columns[12], count_df.columns[13]], 1)
            temp_df = df.merge(count_df, how="left")
            
            #set up so that there is one user/date pair for each day, summing up the time in each HR zone for all activities
            day_groups = temp_df.groupby(['user_id', 'date_md'])
            day_sums = day_groups.agg(np.sum).reset_index()
            day_sums = day_sums.drop(['sport_id', 'intensity_score', 'end_epoch', 'race_period', 'activity_count'], 1)
            
            #bring in the race periods into new dataframe
            race_periods = temp_df.drop(['start', 'end', 'sport_id', 'name', 'intensity_score', 'z1', 'z2', 
                                        'z3', 'z4', 'z5', 'end_epoch'], 1)
            race_periods = race_periods.drop_duplicates()
            new_df = day_sums.merge(race_periods, how='left')
            return_df = new_df
                        
        elif version=="bike":
            #same process, but using the bike start dates
            date_markers = date_markers[date_markers['bike_period_start'] > 0].reset_index()
            bike_start_dict = {}
            for i in range(0, len(date_markers)):
                bike_start_dict[date_markers.values[i][4]] = date_markers.values[i][2]
            df['bike_period'] = df.apply(add_bike, axis=1)
            df['date_md']=df['end'].str[5:10]
            df=df.sort(['user_id', 'end_epoch', 'bike_period'], ascending=[True, True, False])
            groups = df.groupby(['user_id','date_md'])
            things = groups.agg(['count']).join(pd.DataFrame(groups.size(),columns=['activity_count']))
            count_df = pd.DataFrame(things).reset_index()
            count_df = count_df.drop([count_df.columns[2], count_df.columns[3], count_df.columns[4], count_df.columns[5],
                count_df.columns[6], count_df.columns[7], count_df.columns[8], count_df.columns[9], count_df.columns[10],
                count_df.columns[11], count_df.columns[12], count_df.columns[13]], 1)
            temp_df = df.merge(count_df, how="left")
            day_groups = temp_df.groupby(['user_id', 'date_md'])
            day_sums = day_groups.agg(np.sum).reset_index()
            day_sums = day_sums.drop(['sport_id', 'intensity_score', 'end_epoch', 'race_period', 'activity_count'], 1)
            race_periods = temp_df.drop(['start', 'end', 'sport_id', 'name', 'intensity_score', 'z1', 'z2', 
                            'z3', 'z4', 'z5', 'end_epoch'], 1)
            race_periods = race_periods.drop_duplicates()
            new_df = day_sums.merge(race_periods, how='left')
            return_df = new_df
            
        return return_df

In [75]:
test = add_periods(date_markers=date_markers, df=workouts, buildup_days=0, version="race")
test = test.sort(['user_id', 'race_period', 'date_md'], ascending=['True', 'True', 'True'])
test.head(10)

,user_id,date_md,z1,z2,z3,z4,z5,race_period,activity_count
0,828,09-15,1066,1810,1432,4668,679,1,1
1,828,09-19,506,601,1861,696,22,1,1
2,828,09-20,929,1302,1329,5353,321,1,1
3,828,09-21,663,1927,1377,2781,28,1,2
4,828,09-22,767,992,1291,806,1323,1,1
5,828,09-23,268,512,703,1452,89,1,2
6,828,09-25,1453,1601,1803,2406,630,1,1
7,828,09-26,538,353,557,2124,365,2,1
8,828,09-27,423,281,133,1604,1,2,1
9,828,09-28,729,864,2118,2543,67,2,2


In [70]:
groups = test.groupby(['user_id', 'race_period']).cumcount().reset_index()

In [69]:
buildup_days = groups.join(pd.DataFrame(groups.size(), columns='buildup_days'))

,index,0
0,0,0
1,1,1
2,2,2
3,3,3
4,4,4
5,5,5
6,6,6
7,7,0
8,8,1
9,9,2
